The theory behind this example can be found in Greiner Relativistic Quantum Mechanics book

This has been summarized by a blog post "https://physicspages.com/Greiner%20RQM.html" 

I have reproduced the calculations which can be found at "../docs/SKGCP/Ex1_10.pdf"

- There is a discrepecy however with the differential equation I find compared to Greiner, however the results dont change all that much, and as such I have taken Greiners word for it and assume I have made a mistake in the calculation 
- pg(6) of my calcs

# Solution of KG equation for the potential of a Homogenously Charged Sphere

## Problem
Solve the KG equation for a sphere with radius a, such that the potential inside the sphere is give as:

$$eA_0(r) = -\frac{Ze^2}{2a}\left( 3 - \frac{r^2}{a^2}\right) \hspace{3cm} r\leq a$$

In [59]:
using Plots, Roots, QuadGK
plotly()

Plots.PlotlyBackend()

In [3]:
include("../../Constants/src/Short.jl")
m0 = 139.577;

In [4]:
function R1(r,l; a=0.7, N=10, b0=1, E=m0, kw...)
    Z = try Z == Int catch nothing end == nothing ? 10 : Z
    S = Float64[]
    A = E/(ħ*c) + (3Z*α)/(2a)
    B = A^2 - (m0*c)^2/ħ^2
    C = (Z*α)/(2a^3)
    
    push!(S, b0)
    push!(S, -((B*S[1])*(r^2))/(2*(2l+2)))
    push!(S, -((B*S[2] - 2A*C*S[1])*(r^4))/(2*(4l+10)))
    push!(S, -((B*S[3] - 2A*C*S[2] + C^2*S[1])*(r^6))/(2*(6l+21)))
    
    for ni in 5:N
        push!(S, -((B*S[ni-1] - 2A*C*S[ni-2] + C^2*S[ni-3])*(r^(2*(ni-1))))/(2*(6l+21)))
    end
    return r^(l+1)*sum(S)
end

R1 (generic function with 1 method)

In [5]:
plot([r for r in 0.01:0.01:0.9], [a for a in 0.5:0.1:1.2], (r,a) -> R1(r,1; a=a), legend=false, 
    background_color="darkgrey", st=:surface, zlims=(-0.5,0.1))

<!DOCTYPE html>
 
 
 Plots.jl

In [98]:
include("../src/KGCoulomb.jl")

What is the mass of the particle you are dealing with? stdin> 139


R

In [86]:
n = 1
l = 0
a = 0.9
Z = 68
plot([r for r in 0.01:0.01:0.93], [a for a in 0.7:0.1:a], (r,a) -> R1(r,l; a=a, Z=Z), legend=false, 
    background_color="darkgrey", st=:surface)
plot!([r for r in 0.89:0.001:2], [a for a in 0.7:0.1:a], (r,a) -> R(r,n,1; Z=Z)/2e9, st=:surface, 
    background_color="darkgrey")

<!DOCTYPE html>
 
 
 Plots.jl

In order to determine allowed energies levels, we need to impose that the wavefunction and its first derivatives a re continuous at r=a

Given the series for the confluent hypergeometric series, finding the derivative is fairly trivial, we just omit the first term from the sum

In [8]:
function df(r,n,l; N=10, a0=1, kw...)
    try Z == Int catch nothing end == nothing ? Z = 10 : Z = Z
    S = Float64[]
    a = (2Z*α*ε(n,l; kw...))/(ħ*c*β(n,l; kw...))
    b = 2*μ(Z) + 1

    push!(S, a0)

    for i in 2:N
        push!(S, S[i-1]*((a+i-1)*ρ(r,n,l; kw...))/(b+i-1))
    end
    return sum(S[2:end])
end

df (generic function with 1 method)

In [13]:
df(0.1,1,0)

24.86375294954377

In [9]:
function dR(r,n,l; kw...)
    return ρ(r,n,l; kw...)^(ν(Z))*exp(-ρ(r,n,l; kw...)/2)*
        (f(r,n,l; kw...)*((μ(Z)+1/2)/ρ(r,n,l; kw...) - 1/2) + df(r,n,l; kw...))*β(n,l; kw...)
end

dR (generic function with 1 method)

In [14]:
dR(0.1,1,0)

1137.2209447790183

In [34]:
function dR1(r,l; a=0.7, N=10, b0=1, E=m0, kw...)
    Z = try Z == Int catch nothing end == nothing ? 10 : Z
    S = Float64[]
    A = E/(ħ*c) + (3Z*α)/(2a)
    B = A^2 - (m0*c)^2/ħ^2
    C = (Z*α)/(2a^3)
    
    push!(S, b0 * (l+1))
    push!(S, -((B*S[1])*(r^2))/(2*(2l+2)) * (2+l+1))
    push!(S, -((B*S[2] - 2A*C*S[1])*(r^4))/(2*(4l+10)) * (4+l+1))
    push!(S, -((B*S[3] - 2A*C*S[2] + C^2*S[1])*(r^6))/(2*(6l+21)) * (6+l+1))
    
    for ni in 5:N
        push!(S, -((B*S[ni-1] - 2A*C*S[ni-2] + C^2*S[ni-3])*(r^(2*(ni-1))))/(2*(6l+21)) * (2*ni+l+1))
    end
    return r^(l)*sum(S)
end

dR1 (generic function with 1 method)

In [35]:
dR1(0.1,0)

-1.0222786714530034

In [ ]:
function ψ()
    for r < a 
        return R1(r,l; a=r, N=10, b0=1, E=m0, kw...)

In [93]:
let 
    F(r,n,l; N=10, b0=1, E=m0, kw...) = R(r,n,l; kw...)*(dR1(r,l; N=N, b0=b0, E=E, kw...) - 
        (μ(Z) + 1/2)* R1(r,l; N=N, b0=b0, E=E, kw...)) / (R1(r,l; N=N, b0=b0, E=E, kw...) * 
            (dR(r,n,l; kw...) - R(r,n,l; kw...)/2))
    Enew(r,n,l; kw...) = sqrt(m0^2*c^4 - (ħ*c*F(r,n,l; kw...)/2)^2)
    Eold = m0
    eps, E = 2, 0
    while eps > 0.0001
        E = Enew(0.7, 1, 0; E=Eold)
        eps = E - Eold
        Eold = E
    end
    E
end

138.9992622275224

In [117]:
let 
    F(r,n,l; a=r, N=10, b0=1, E=m0, kw...) = (dR(r,n,l; kw...) / R(r,n,l; kw...)) - 
        (dR1(r,l; a=a, N=N, b0=b0, E=E, kw...) / R1(r,l; a=a, N=N, b0=b0, E=E, kw...))
    A = []
#     n = 2
#     l = 2
    n = 1
    l = 0
    push!(A, fzeros(E -> F(0.8,n,l; E=E, Z=30), -m0, m0))
    println("The energy values are ", A)
end

The energy values are Any[[138.80473223415774, 138.8963696040093, 138.89749389738566, 138.9929682501171]]


In [101]:
ε(1,0; E=134)

MethodError: MethodError: no method matching ε(::Int64, ::Int64; E=134)
Closest candidates are:
  ε(::Any, ::Any; Z, m0) at /Users/joe/Julia/mypackages/QMBlog/src/KGCoulomb.jl:7 got unsupported keyword argument "E"